# Projet de Machine Learning for data sciences
Utilisation des données internet sur les articles publiés sur https://news.bitcoin.com/ et les données marché pour prédire la montée ou la descente du BitCoin.  

## Import des packages
Les fonctions créées pour le projet seront expliquée lors de l'utilisation.

In [1]:
import pandas as pd #pandas
import data_collecting.api_bitcoin as bitcoin_api #data making
from importlib import reload
from datetime import datetime,timedelta
from data_processing.from_html_to_txt import extract_content_from_html as ext_html
import os

Il faut lancer web_knowledge.scrapper sur l'invite de commande (ou utiliser les données déjà récoltées). 
        ==> dossier wayback_dowloads
> python web_knowledge/scrapper.py 'https://news.bitcoin.com'

In [2]:
#Récupération des données de binance :
reload(bitcoin_api)
data = bitcoin_api.get_historical_data(start_date='2017-08-17') # On est limité à un interval de 1 semaine pour s'accorder aux données web

## Données disponibles grâce à l'appel API binance :
- Open time: The timestamp (in milliseconds) indicating when the candlestick period starts.
- Open: The opening price of the asset for the candlestick period.
- High: The highest price of the asset during the candlestick period.
- Low: The lowest price of the asset during the candlestick period.
- Close: The closing price of the asset for the candlestick period.
- Volume: The trading volume of the asset for the candlestick period.
- Close time: The timestamp (in milliseconds) indicating when the candlestick period ends.
- Quote asset volume: The trading volume of the quote asset for the candlestick period.
- Number of trades: The number of trades that occurred during the candlestick period.
- Taker buy base asset volume: The volume of the base asset involved in taker trades during the candlestick period.
- Taker buy quote asset volume: The volume of the quote asset involved in taker trades during the candlestick period.
- Ignore: This column is typically ignored and contains no relevant data.

In [3]:
columns=["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades","Taker buy base","Taker buy quote","Ignore"]
Binance_dataframe=pd.DataFrame(data,columns=columns)

### On va maintenant organiser nos données pour permettre la jointure avec les données web

In [4]:
open_time=[datetime.strftime(datetime.fromtimestamp(i/1000),format='%Y%m%d') for i in Binance_dataframe["Open time"]]
Binance_dataframe["Open time"]=open_time

In [5]:
#Sauvegarde des données
Binance_dataframe.to_csv("data/market/Binance_data.csv",index=False)

### On va maintenant convertir nos fichiers html en txt (formaté)

In [6]:
files = os.listdir("wayback_downloads")
log=''
for i in files:
    already_done = os.listdir("data/html/")
    if f'{i.split(".")[0][:8]}.txt' not in already_done:
        try:
            ext_html("wayback_downloads/" + i, ['div', 'p', 'h6', 'h4'], f'data/html/{i.split(".")[0][:8]}.txt')
        except Exception as e:
            log+=f"Failed to process {i}: {e}\n"
            with open("log.txt","w") as f: f.write(log)

## Création du dataframe pour ACM
On utilise nos fichiers HTML formaté en txt pour compter les mots

In [7]:
from data_processing.text_processing import text_processing as make_csv

c:\Users\lenovo\Desktop\Cours\35_ML4DS\.env_mlds\Lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\lenovo\Desktop\Cours\35_ML4DS\.env_mlds\Lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\lenovo\Desktop\Cours\35_ML4DS\.env_mlds\Lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\lenovo\Desktop\Cours\35_ML4DS\.env_mlds\Lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


ImportError: cannot import name 'text_processing' from 'data_processing.text_processing' (c:\Users\lenovo\Desktop\Cours\35_ML4DS\data_processing\text_processing.py)

Ici on lemmatise notre texte pour qu'il ne contienne que les mots importants (pas générique) du texte.
Réutilisation d'un script fait en Analyse textuelle

In [ ]:
word_count=make_csv()
word_count.index = [i[:8] for i in os.listdir("data/html/")]

TreeTaggerError: TreeTagger parameter file invalid: english.par

In [18]:
word_count.to_csv("data/temp/word_count.csv")
word_count=pd.read_csv("data/temp/word_count.csv",index_col=0)
word_count=word_count.T

In [ ]:
to_drop=[]
for i in range(len(word_count.index)):
    cache=word_count.iloc[i].value_counts()
    cache=pd.DataFrame(data=cache.to_numpy().flatten(),index=cache.index.to_numpy().flatten(),columns=["count"])
    if 0 not in cache.index:
        to_drop.append(word_count.index[i])
    elif int(cache.iloc[0])>len(word_count.columns)*0.5:
        to_drop.append(word_count.index[i])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5480\3963066350.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  elif int(cache.iloc[0])>len(word_count.columns)*0.5:


In [20]:
len(word_count.index)-len(to_drop)

100

On garde alors XXX mots sur les plus de 9000 existants ==> Analyse globale permettant clustering

In [21]:
word_count.drop(to_drop,inplace=True)
word_count.to_csv("data/temp/word_count.csv") #check point

On formate les données extraite des pages html pour n'avoir qu'un enregistrement par semaine pour s'accorder aux données de marché.

In [ ]:
word_count=pd.read_csv("data/temp/word_count.csv",index_col=0)
word_count=word_count.T
word_count['Open time']=0
Binance_dataframe=pd.read_csv("data/market/Binance_data.csv")
word_count.index=[int(i) for i in word_count.index]
index_to_keep=[]
for i in range(len(Binance_dataframe['Open time'])):
    borne=int(datetime.strftime(datetime.strptime(str(Binance_dataframe["Open time"][i]),'%Y%m%d'),format='%Y%m%d'))
    cache=max(word_count.index[(word_count.index<=borne)])
    if cache not in index_to_keep:
        index_to_keep.append(cache)
        word_count['Open time'][cache]=Binance_dataframe['Open time'][i]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5480\1212387663.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  word_count['Open time'][cache]=Binance_dataframe['Open time'][i]
C:\Users\lenovo\AppData\Local\Temp\ipykernel_5480\1212387663.p

In [23]:
word_count.loc[index_to_keep].to_csv("data/temp/word_count_indexed.csv") #TO ACM

Veuillez lancer le script R dans "data_processing/clustering.r"

In [107]:
len(os.listdir("data/html/"))

410

In [29]:
len(word_count[word_count['Open time']!=0])

573

In [38]:
to_merge=pd.read_csv("data/temp/cluster_web.csv",index_col=0)
to_merge['Open time']=word_count['Open time'].values
final_data=pd.merge(Binance_dataframe,to_merge[['Open time','clust']],on='Open time',how='left')
final_data.to_csv("data/final_data.csv",index=False) #TO MODELISATION

In [ ]:
final_data

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base,Taker buy quote,Ignore,clust
0,20170817,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06,0,2
1,20170823,4040.00,4265.80,4013.89,4114.01,1001.136565,1503532799999,4.148686e+06,8629,309.419092,1.293567e+06,0,1
2,20170830,4564.52,4647.51,4416.01,4555.14,808.468771,1504137599999,3.683715e+06,5551,196.062252,8.946202e+05,0,1
3,20170902,4796.16,4939.19,4286.87,4472.14,929.148595,1504396799999,4.257724e+06,7475,379.199863,1.738334e+06,0,1
4,20170903,4508.50,4714.76,4298.33,4509.08,691.216198,1504483199999,3.121053e+06,6022,258.934536,1.169645e+06,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,20201115,16069.56,16180.00,15774.72,15957.00,43596.841513,1605484799999,6.966517e+08,854441,20853.270846,3.332391e+08,0,3
569,20201116,15957.00,16880.00,15864.00,16713.57,81300.675924,1605571199999,1.335728e+09,1202386,41370.213659,6.795934e+08,0,3
570,20201117,16713.08,17858.82,16538.00,17659.38,115221.403102,1605657599999,1.982752e+09,1646652,58262.062597,1.003081e+09,0,3
571,20201118,17659.38,18476.93,17214.45,17776.12,149019.788134,1605743999999,2.663338e+09,2091832,73412.736278,1.312369e+09,0,3
